In [1]:
import pandas as pd
import numpy as np
import scipy.optimize
import os, sys
import json
import csv
import re

In [ ]:
'''
We need:
 - COSTS (of the different commodities)
 - X is an array of the weights of the commodities (positions sorted by name)
 - A will be the chemistry matrix of the commodities (m x n, m = elements, n = commodities in sorted order)
 - Y is a (m x 2 array of the min and max allowed of each element in the steel)
 - TOTAL_WEIGHT The total charge weight.
'''


In [398]:
TOTAL_WEIGHT = 100  # so all commodity weights effectively becomes a pct

In [399]:
COSTS = pd.read_csv('data/steel_university/cost.csv')

In [400]:
COSTS.set_index('commodity', inplace=True)
COSTS.sort_index(inplace=True)

In [402]:
COSTS

,price
commodity,
Direct_Reduced_Iron,220
EAF_Dust,-120
Internal_Low_Alloyed,240
No1_Bundles,180
No1_Heavy,160
No2_Bundles,170
No2_Heavy,140
Plate_And_Structural,290
Shredded,200


In [461]:
f = open('data/steel_university/chemistry.csv')
csv_f = csv.reader(f)

In [462]:
element_chemistry_data = [l for l in csv_f]

In [405]:
#element_chemistry_data

In [463]:
re_1 = re.compile('([0-9.]+)%(\w+)')
def create_element_chemistry_df(d=element_chemistry_data):
    d2 = {}
    all_elements = set()
    for l in d:
        if not l: continue
        #print(l[1:])
        d2[l[0]] = dict(map(lambda x: re_1.match(x.strip()).groups()[::-1], l[1:]))  # Ha :)
        all_elements.update(d2[l[0]].keys())
    cols = list(d2.keys())
    cols.sort()
    df1 = pd.DataFrame(d2, index=all_elements)
    df1.sort_index(inplace=True)
    df1 = df1[cols]
    df1 = df1.astype('float')
    ae = list(all_elements)
    ae.sort()
    df1.fillna(0.0, inplace=True)
    return (df1, ae)


In [464]:
(A, all_elements) = create_element_chemistry_df()

In [465]:
A

,Direct_Reduced_Iron,EAF_Dust,Internal_Low_Alloyed,No1_Bundles,No1_Heavy,No2_Bundles,No2_Heavy,Plate_And_Structural,Shredded,Turnings
C,2.40,0.000,0.1700,0.027,0.025,0.040,0.030,0.250,0.030,0.000
Ca,0.00,0.140,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ce,0.00,0.000,0.0030,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Cr,0.00,20.030,0.2600,0.032,0.200,0.032,0.260,0.150,0.120,0.698
Cu,0.00,0.000,0.0200,0.018,0.180,0.018,0.180,0.000,0.160,0.000
Mn,0.00,4.440,0.3100,0.120,0.000,0.120,0.000,1.000,0.000,0.000
Mo,0.00,0.000,0.1400,0.000,0.030,0.000,0.030,0.050,0.020,0.538
N,0.00,0.000,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Nb,0.03,0.000,0.0010,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ni,0.00,11.200,0.4000,0.020,0.150,0.020,0.180,0.150,0.100,0.000


In [409]:
A.shape

(18, 10)

In [410]:
len(all_elements)

18

In [70]:
all_elements

['C',
 'Ca',
 'Ce',
 'Cr',
 'Cu',
 'Mn',
 'Mo',
 'N',
 'Nb',
 'Ni',
 'P',
 'Pb',
 'S',
 'Si',
 'Sn',
 'Ti',
 'V',
 'W']

In [466]:
def create_min_max(f='data/steel_university/req.csv', all_elements=all_elements):
    elements_min_max_req = pd.read_csv(f)
    elements_min_max_req.set_index('element', inplace=True)
    elements_min_max_req = elements_min_max_req.merge(
        pd.Series(index=all_elements, name='_tmp'), how='outer', left_index=True, right_index=True)
    elements_min_max_req.sort_index(inplace=True)
    elements_min_max_req.drop('_tmp', axis=1, inplace=True)
    return elements_min_max_req


In [467]:
elements_min_max_req = create_min_max()

/Users/nishant.deshpande/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [468]:
elements_min_max_req.shape

(18, 2)

In [469]:
elements_min_max_req

,lb,ub
C,0.1,0.120
Ca,NaN,NaN
Ce,NaN,NaN
Cr,0.0,0.100
Cu,0.0,0.150
Mn,1.0,1.500
Mo,0.0,0.040
N,0.0,0.005
Nb,0.0,0.050
Ni,0.0,0.150


In [470]:
def f1(x, *args):
    # x: np.array of weights of commodities (position according to alphabetical sort of commodity names as for COSTS
    costs = args[0] # pass COSTS like data as the first arg
    return costs.price.dot(x)

In [471]:
def f1p(x, *args):
    costs = args[0]
    return costs.price * 1.0

In [472]:
x = pd.Series(10, index=COSTS.index.values)

In [473]:
v = f1(x, COSTS)

In [474]:
v

15900

In [420]:
COSTS

,price
commodity,
Direct_Reduced_Iron,220
EAF_Dust,-120
Internal_Low_Alloyed,240
No1_Bundles,180
No1_Heavy,160
No2_Bundles,170
No2_Heavy,140
Plate_And_Structural,290
Shredded,200


In [475]:
x

Direct_Reduced_Iron     10
EAF_Dust                10
Internal_Low_Alloyed    10
No1_Bundles             10
No1_Heavy               10
No2_Bundles             10
No2_Heavy               10
Plate_And_Structural    10
Shredded                10
Turnings                10
dtype: int64

In [422]:
print(A.shape, x.shape)

(18, 10) (10,)


In [476]:
A.fillna(0.0, inplace=True)

In [477]:
lb = elements_min_max_req.lb*100
ub = elements_min_max_req.ub*100

In [478]:
ub.fillna(9999999, inplace=True)
lb.fillna(0, inplace=True)

In [479]:
print(pd.DataFrame({'lb':lb, 'ub':ub}))

       lb         ub
C    10.0       12.0
Ca    0.0  9999999.0
Ce    0.0  9999999.0
Cr    0.0       10.0
Cu    0.0       15.0
Mn  100.0      150.0
Mo    0.0        4.0
N     0.0        0.5
Nb    0.0        5.0
Ni    0.0       15.0
P     0.0        2.0
Pb    0.0  9999999.0
S     0.0        3.0
Si   10.0       30.0
Sn    0.0  9999999.0
Ti    0.0        1.0
V     0.0  9999999.0
W     0.0  9999999.0


In [480]:
print(lb.shape, ub.shape)

(18,) (18,)


In [481]:
constraint1 = scipy.optimize.LinearConstraint(A=A, lb=lb, ub=ub)
#constraint1 = scipy.optimize.LinearConstraint(A=A, lb=lb, ub=lb)


In [482]:
print(lb.shape, ub.shape, A.shape)

(18,) (18,) (18, 10)


In [484]:
o1 = A[0:0]

In [485]:
o1

,Direct_Reduced_Iron,EAF_Dust,Internal_Low_Alloyed,No1_Bundles,No1_Heavy,No2_Bundles,No2_Heavy,Plate_And_Structural,Shredded,Turnings


In [486]:
o1.loc['weight_factor'] = 1

/Users/nishant.deshpande/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [487]:
print(o1.shape, x.shape)

(1, 10) (10,)


In [488]:
o1.dot(x)

weight_factor    100.0
dtype: float64

In [489]:
constraint2 = scipy.optimize.LinearConstraint(A=o1, lb=100, ub=100)

In [490]:
print(x.shape, o1.shape, A.shape, ub.shape, lb.shape)

(10,) (1, 10) (18, 10) (18,) (18,)


In [491]:
x_bounds = [(0.0,TOTAL_WEIGHT*1.0) for e in x]

In [492]:
x_bounds

[(0.0, 100.0),
 (0.0, 100.0),
 (0.0, 100.0),
 (0.0, 100.0),
 (0.0, 100.0),
 (0.0, 100.0),
 (0.0, 100.0),
 (0.0, 100.0),
 (0.0, 100.0),
 (0.0, 100.0)]

In [493]:
b1 = scipy.optimize.Bounds(lb=np.array([0]*10), ub=np.array([TOTAL_WEIGHT*1.0]*10))

In [494]:
def cb1(xk, state):
    print('xk: ', xk)
    print('state.x: ', state.x)

In [495]:
def cb2(xk):
    print('xk: ', xk)


In [496]:
res = scipy.optimize.minimize(fun=f1, x0=x, args=(COSTS,), method='SLSQP', # method='trust-constr', 
                              jac=f1p,
                              constraints=[constraint1, constraint2],
                             bounds=b1, callback=cb2)

xk:  [2.06265153e+00 9.92251670e+00 2.52191157e-10 1.11130320e+01
 6.96216154e+00 2.13601514e+01 2.44771137e+01 1.19472438e+01
 1.07347020e-10 1.21551293e+01]
xk:  [ 2.01815746  9.91072335  0.          5.18748322  0.         28.29191025
 30.49894533 11.87886109  0.         12.2139193 ]
xk:  [ 1.98766498  9.90829289  0.          0.          0.         33.35315772
 30.6425848  11.90480069  0.         12.20349893]
xk:  [ 1.98766498  9.9082929   0.          0.          0.         33.35315773
 30.64258481 11.90480069  0.         12.20349893]
xk:  [ 1.98766498  9.90829289  0.          0.          0.         33.35315772
 30.64258481 11.90480069  0.         12.20349893]
xk:  [ 1.98766498  9.90829289  0.          0.          0.         33.35315773
 30.64258481 11.90480069  0.         12.20349893]
xk:  [ 1.98766467  9.90829325  0.          0.          0.         33.35315809
 30.64258492 11.9048004   0.         12.20349897]
xk:  [ 1.98766182  9.90829605  0.          0.          0.         33.3531

In [497]:
res

     fun: 14003.066857167223
     jac: array([ 220., -120.,  240.,  180.,  160.,  170.,  140.,  290.,  200.,
        110.])
 message: 'Positive directional derivative for linesearch'
    nfev: 122
     nit: 19
    njev: 15
  status: 8
 success: False
       x: array([ 1.98766493,  9.90829305,  0.        ,  0.        ,  0.        ,
       33.35315773, 30.64258478, 11.90480059,  0.        , 12.20349895])

In [498]:
A

,Direct_Reduced_Iron,EAF_Dust,Internal_Low_Alloyed,No1_Bundles,No1_Heavy,No2_Bundles,No2_Heavy,Plate_And_Structural,Shredded,Turnings
C,2.40,0.000,0.1700,0.027,0.025,0.040,0.030,0.250,0.030,0.000
Ca,0.00,0.140,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ce,0.00,0.000,0.0030,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Cr,0.00,20.030,0.2600,0.032,0.200,0.032,0.260,0.150,0.120,0.698
Cu,0.00,0.000,0.0200,0.018,0.180,0.018,0.180,0.000,0.160,0.000
Mn,0.00,4.440,0.3100,0.120,0.000,0.120,0.000,1.000,0.000,0.000
Mo,0.00,0.000,0.1400,0.000,0.030,0.000,0.030,0.050,0.020,0.538
N,0.00,0.000,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Nb,0.03,0.000,0.0010,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ni,0.00,11.200,0.4000,0.020,0.150,0.020,0.180,0.150,0.100,0.000


In [499]:
COSTS.shape

(10, 1)

In [500]:
sol1 = COSTS.copy()

In [501]:
sol1

,price
commodity,
Direct_Reduced_Iron,220
EAF_Dust,-120
Internal_Low_Alloyed,240
No1_Bundles,180
No1_Heavy,160
No2_Bundles,170
No2_Heavy,140
Plate_And_Structural,290
Shredded,200


In [502]:
sol1['weights']= res.x

In [503]:
sum(res.x)

100.00000002486016

In [504]:
res.x.shape

(10,)

In [505]:
round(sol1,2)

,price,weights
commodity,,
Direct_Reduced_Iron,220,1.99
EAF_Dust,-120,9.91
Internal_Low_Alloyed,240,0.00
No1_Bundles,180,0.00
No1_Heavy,160,0.00
No2_Bundles,170,33.35
No2_Heavy,140,30.64
Plate_And_Structural,290,11.90
Shredded,200,0.00


In [506]:
heat_chem = A.dot(sol1.weights)

In [507]:
results1 = pd.DataFrame({'lb':lb, 'ub':ub, 'r1':heat_chem})

In [508]:
results1['compliant'] = results1.apply(lambda r: r['r1'] >= r['lb'] and r['r1'] <= r['ub'], axis=1)

In [509]:
results1.sort_index()

,lb,ub,r1,compliant
C,10.0,12.0,10.000000,False
Ca,0.0,9999999.0,1.387161,True
Ce,0.0,9999999.0,0.000000,True
Cr,0.0,10.0,217.801245,False
Cu,0.0,15.0,6.116022,True
Mn,100.0,150.0,59.900001,False
Mo,0.0,4.0,8.080000,False
N,0.0,0.5,0.000000,True
Nb,0.0,5.0,0.059630,True
Ni,0.0,15.0,118.941331,False


In [279]:
A

,Direct_Reduced_Iron,EAF_Dust,Internal_Low_Alloyed,No1_Bundles,No1_Heavy,No2_Bundles,No2_Heavy,Plate_And_Structural,Shredded,Turnings
C,2.40,0.0000,0.1700,0.027,0.025,0.040,0.030,0.250,0.030,0.000
Ca,0.00,0.1400,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ce,0.00,0.0000,0.0030,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Cr,0.00,0.2003,0.2600,0.032,0.200,0.032,0.260,0.150,0.120,0.698
Cu,0.00,0.0000,0.0200,0.018,0.180,0.018,0.180,0.000,0.160,0.000
Mn,0.00,0.0444,0.3100,0.120,0.000,0.120,0.000,1.000,0.000,0.000
Mo,0.00,0.0000,0.1400,0.000,0.030,0.000,0.030,0.050,0.020,0.538
N,0.00,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Nb,0.03,0.0000,0.0010,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ni,0.00,0.1120,0.4000,0.020,0.150,0.020,0.180,0.150,0.100,0.000


In [123]:
res.fun

14003.066857167223

In [133]:
COSTS.price.dot(res.x)

14003.066857167223

In [134]:
A[0:0]

,Direct_Reduced_Iron,EAF_Dust,Internal_Low_Alloyed,No1_Bundles,No1_Heavy,No2_Bundles,No2_Heavy,Plate_And_Structural,Shredded,Turnings


In [136]:
b2_lb = np.array([0]*10)
b2_ub = np.array([TOTAL_WEIGHT]*10)
b2_ub[1] = 0 # We don't have any EAF dust
b2 = scipy.optimize.Bounds(lb=b2_lb, ub=b2_ub)

In [137]:
b2

Bounds(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([100,   0, 100, 100, 100, 100, 100, 100, 100, 100]))

In [186]:
res2 = scipy.optimize.minimize(fun=f1, args=(COSTS,), x0=x, method='SLSQP', # method='trust-constr', 
                              jac=f1p,
                              constraints=[constraint1, constraint2],
                             bounds=b2, callback=cb2)

xk:  [ 2.1621013   0.          0.         14.85001163  5.11871154 25.06845015
 20.06182066 10.70978459  0.         12.02912014]
xk:  [ 2.13159726  0.          0.          9.46921011  0.         31.890109
 23.95577034 10.5368817   0.         12.01643158]
xk:  [2.05315069e+00 0.00000000e+00 9.23689880e-09 0.00000000e+00
 4.12034647e-09 3.93851286e+01 2.67862357e+01 1.07737846e+01
 7.20640891e-09 1.10017005e+01]
xk:  [ 2.05315068  0.          0.          0.          0.         39.3851286
 26.7862357  10.77378457  0.         11.00170046]
xk:  [ 2.05315068  0.          0.          0.          0.         39.3851286
 26.7862357  10.77378457  0.         11.00170046]


In [187]:
print(res2.fun)
print(res2.x)

15231.822587144256
[ 2.05315068  0.          0.          0.          0.         39.3851286
 26.7862357  10.77378457  0.         11.00170046]


In [144]:
print(res.fun)
print(res.x)

14003.066857167223
[ 1.98766493  9.90829305  0.          0.          0.         33.35315773
 30.64258478 11.90480059  0.         12.20349895]


In [145]:
(res2.fun - res.fun)/res.fun

0.08774904401374871

In [151]:
# Just not having 1 commodity (EAF Dust) results in a 8.7% increase in cost!
# The recipes are actually fairly similar, but cost is a lot more.

# Well thats because there is -ve cost for EAF Dust :)

In [156]:
COSTS2 = COSTS.copy()

In [162]:
type(COSTS2)

pandas.core.frame.DataFrame

In [177]:
COSTS2.at['EAF_Dust', 'price'] = 0

In [178]:
COSTS2

,price
commodity,
Direct_Reduced_Iron,220.0
EAF_Dust,0.0
Internal_Low_Alloyed,240.0
No1_Bundles,180.0
No1_Heavy,160.0
No2_Bundles,170.0
No2_Heavy,140.0
Plate_And_Structural,290.0
Shredded,200.0


In [188]:
res = scipy.optimize.minimize(fun=f1, args=(COSTS2,), x0=x, method='SLSQP', # method='trust-constr', 
                              jac=f1p,
                              constraints=[constraint1, constraint2],
                             bounds=b1, callback=cb2)

xk:  [2.05288628e+00 9.91635055e+00 1.02730269e-10 1.07552725e+01
 7.12581932e+00 2.10057320e+01 2.49761682e+01 1.20600830e+01
 3.26547678e-11 1.21076881e+01]
xk:  [2.00800398e+00 9.90431248e+00 6.29832370e-09 4.80387906e+00
 4.56336302e-10 2.79897054e+01 3.11363967e+01 1.19896224e+01
 4.86433174e-09 1.21680799e+01]
xk:  [ 1.97976636  9.90206173  0.          0.          0.         32.67668362
 31.26941437 12.01364387  0.         12.15843007]
xk:  [ 1.97976636  9.90206173  0.          0.          0.         32.67668362
 31.26941437 12.01364387  0.         12.15843007]


In [189]:
res.x

array([ 1.97976636,  9.90206173,  0.        ,  0.        ,  0.        ,
       32.67668362, 31.26941437, 12.01364387,  0.        , 12.15843007])

In [190]:
res.fun

15189.686853261048

In [191]:
res2 = scipy.optimize.minimize(fun=f1, args=(COSTS2,), x0=x, method='SLSQP', # method='trust-constr', 
                              jac=f1p,
                              constraints=[constraint1, constraint2],
                             bounds=b2, callback=cb2)

xk:  [2.16210131e+00 0.00000000e+00 4.55764315e-09 1.48500116e+01
 5.11871154e+00 2.50684501e+01 2.00618207e+01 1.07097846e+01
 1.42404311e-09 1.20291201e+01]
xk:  [2.13159727e+00 0.00000000e+00 6.75974555e-09 9.46921012e+00
 1.30200206e-09 3.18901090e+01 2.39557703e+01 1.05368817e+01
 5.17095664e-09 1.20164316e+01]
xk:  [2.05315069e+00 0.00000000e+00 1.26292727e-08 0.00000000e+00
 5.14038599e-09 3.93851286e+01 2.67862357e+01 1.07737846e+01
 9.65754002e-09 1.10017004e+01]
xk:  [ 2.05315068  0.          0.          0.          0.         39.38512862
 26.78623569 10.77378459  0.         11.00170045]
xk:  [ 2.05315068  0.          0.          0.          0.         39.38512862
 26.78623569 10.77378459  0.         11.00170045]


In [192]:
res2.fun

15231.822593249908

In [193]:
res2.x

array([ 2.05315068,  0.        ,  0.        ,  0.        ,  0.        ,
       39.38512862, 26.78623569, 10.77378459,  0.        , 11.00170045])